In [1]:
from diffusers.models.attention import BasicTransformerBlock
from fastcore.basics import patch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

@patch
def forward(self: BasicTransformerBlock, x, context=None):
    # x = self.attn1(self.norm1(x)) + x
    x = self.attn1(self.norm1(x.contiguous())) + x  # <--- added x.contiguous()
    x = self.attn2(self.norm2(x), context=context) + x
    x = self.ff(self.norm3(x)) + x
    return x


model_id = "stabilityai/stable-diffusion-2-1"
model2_id = 'CompVis/stable-diffusion-v1-4'
DEVICE = 'mps'

pipe = StableDiffusionPipeline.from_pretrained(
    model2_id).to(DEVICE)

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
# pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

image.save("astronaut_rides_horse.png")


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

  0%|          | 0/50 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/2b/wkpwqypx6fz7k9s29h718tkr0000gn/T/ipykernel_45527/1409884852.py:28 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/2b/wkpwqypx6fz7k9s29h718tkr0000gn/T/ipykernel_45527/1409884852.py'                 │
│                                                                                                  │
│ /Users/graziano/miniconda3/envs/tf/lib/python3.9/site-packages/torch/autograd/grad_mode.py:27 in │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /Users/graziano/miniconda3/envs/tf/lib/python3.9/site-packages/diffusers/pipelines/stable_diffus │
│ ion/pipeline_stable_diffusion.py:529 in __call__                                                 │
│                                                                                                  │
│   526 │   │   │   │   latent_model_input = self.scheduler.scale_model_input(latent_model_input   │
│   527 │   │   │   │                                                                              │
│   528 │   │   │   │   # predict the noise residual                                               │
│ ❱ 529 │   │   │   │   noise_pred = self.unet(latent_model_input, t, encoder_hidden_states=text   │
│   530 │   │   │   │                                                                              │
│   531 │   │   │   │   # perform guidance                                                         │
│   532 │   │   │   │   if do_classifier_free_guidance:                                            │
│                                                                                                  │
│ /Users/graziano/miniconda3/envs/tf/lib/python3.9/site-packages/torch/nn/modules/module.py:1194   │
│ in _call_impl                                                                                    │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /Users/graziano/miniconda3/envs/tf/lib/python3.9/site-packa